In [1]:
import numpy as np
import torch 
from torch import nn
from torch.utils.data import DataLoader,random_split
from torchvision import transforms,datasets
from torch import optim

In [2]:
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((256,256),),
    transforms.Normalize((0.5,),(0.5,))
])

In [3]:
root=r"C:\Users\saipr\Downloads\archive\PotatoPlants"
dataset=datasets.ImageFolder(root=root,transform=transform)
train_size=int(0.8*(len(dataset)))
test_size=len(dataset)-train_size
train_dataset,test_dataset=random_split(dataset,[train_size,test_size])
train_loader=DataLoader(train_dataset,batch_size=64,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=64,shuffle=True)
class_names=dataset.classes
class_idx=dataset.class_to_idx
print(class_idx)

{'Potato___Early_blight': 0, 'Potato___Late_blight': 1, 'Potato___healthy': 2}


In [47]:
class CNN(nn.Module):
    def __init__(self,input_size=256*256,output_size=len(class_names)):
        super().__init__()
        self.network=nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=4,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=4,out_channels=8,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,padding=1,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=32,out_channels=32,stride=1,kernel_size=3,padding=1),
            nn.Flatten(),
            nn.Linear(8*8*32,64),
            nn.ReLU(),
            nn.Linear(64,output_size)
            #putting relu here give a huge disaster like no training in last 2 classes
        )
    def forward(self,x):
        return self.network(x)

In [48]:
model=CNN()
criterion=nn.CrossEntropyLoss()
optimiser=optim.Adam(model.parameters(),lr=0.001)

In [50]:
num_epochs=20
epoch=0
for epoch in range(num_epochs):
    for input,output in train_loader:
        pred_output=model(input)
        loss=criterion(pred_output,output)
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()
    print(f'Loss={loss} in epoch {epoch+1}')
    epoch+=1
    if loss<0.001 or epoch>30:
        break

Loss=0.1350763738155365 in epoch 1
Loss=0.45409977436065674 in epoch 2
Loss=0.07617770880460739 in epoch 3
Loss=0.3965681195259094 in epoch 4
Loss=0.01378745399415493 in epoch 5
Loss=0.4305189847946167 in epoch 6
Loss=0.11088572442531586 in epoch 7
Loss=0.046532243490219116 in epoch 8
Loss=0.10169071704149246 in epoch 9
Loss=0.06096541881561279 in epoch 10
Loss=0.023595979437232018 in epoch 11
Loss=0.015679046511650085 in epoch 12
Loss=0.043566714972257614 in epoch 13
Loss=0.08324064314365387 in epoch 14
Loss=0.025846906006336212 in epoch 15
Loss=0.01315644197165966 in epoch 16
Loss=0.0027615688741207123 in epoch 17
Loss=0.002291327342391014 in epoch 18
Loss=0.015097427181899548 in epoch 19
Loss=0.018221400678157806 in epoch 20


In [51]:
from sklearn.metrics import classification_report
def accuracy(model,dataloader,classes):
    model.eval()
    with torch.no_grad():
        y_test,y_pred=[],[]
        for input,output in dataloader:
            pred_output=model(input)
            y_test.extend(output.numpy())
            y_pred.extend(torch.argmax(pred_output,dim=1).numpy())
        #classes=dataloader.classes
        cor,tot=[],[]
        for i in range(len(classes)):
            cor.append(sum(1 for j,k in zip(y_test,y_pred) if j==i and k==i))
            tot.append(sum (1 for j in y_test if j==i))
        print(f'accuracy report of {dataloader}:')
        for i,cl in enumerate(classes):
            print(f'{cl}:[{cor[i]}/{tot[i]}]')
        print(classification_report(y_test,y_pred))
    model.train()

print(accuracy(model,test_loader,dataset.classes))
print("this above is test accuracy")
print(accuracy(model,train_loader,dataset.classes))
print("the above is training accuracy")
root_pvp=r"C:\Users\saipr\Downloads\archive\PlantVillage\PlantVillage"
pvp_data=datasets.ImageFolder(root=root_pvp,transform=transform)
pvp_data_loader=DataLoader(pvp_data,batch_size=64,shuffle=True)
print(accuracy(model,pvp_data_loader,pvp_data.classes))
print("the above is pvp accuracy")

accuracy report of <torch.utils.data.dataloader.DataLoader object at 0x000001D8E9D7FE30>:
Potato___Early_blight:[93/96]
Potato___Late_blight:[107/107]
Potato___healthy:[11/13]
              precision    recall  f1-score   support

           0       1.00      0.97      0.98        96
           1       0.96      1.00      0.98       107
           2       1.00      0.85      0.92        13

    accuracy                           0.98       216
   macro avg       0.99      0.94      0.96       216
weighted avg       0.98      0.98      0.98       216

None
this above is test accuracy
accuracy report of <torch.utils.data.dataloader.DataLoader object at 0x000001D8FFC0A360>:
Potato___Early_blight:[883/904]
Potato___Late_blight:[893/893]
Potato___healthy:[109/139]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       904
           1       0.95      1.00      0.97       893
           2       0.99      0.78      0.88       139

    accuracy

In [40]:
y_test_train,y_pred_train=[],[]
model.eval()
with torch.no_grad():
    for input,output in train_loader:
        pred_output=model(input)
        y_test_train.extend(output.numpy())
        y_pred_train.extend(torch.argmax(pred_output,dim=1).numpy())
model.train()
class1=sum(1 for x,y in zip(y_test_train,y_pred_train) if x==0 & y==0)
class2=sum(1 for x,y in zip(y_test_train,y_pred_train) if x==1 & y==1)
class3=sum(1 for x,y in zip(y_test_train,y_pred_train) if x==2 & y==2)
t1=sum(1 for x in y_test_train if x==0)
t2=sum(1 for x in y_test_train if x==1)
t3=sum(1 for x in y_test_train if x==2)
print('Training accuracy:')
print(f'ratio of correctly predicted classes:\n Class1:[{class1}/{t1}]\nClass2:[{class2}/{t2}]\nClass3:[{class3}/{t3}]')

Training accuracy:
ratio of correctly predicted classes:
 Class1:[904/904]
Class2:[858/893]
Class3:[137/139]


In [30]:
# %%
import torch
from torch import nn
from torchvision import datasets,transforms
from torch.utils.data import DataLoader,random_split
from torch import optim
from sklearn.metrics import classification_report

# %%
transform=transforms.Compose([
    transforms.ToTensor(),#tranforms to the tensor
    transforms.Resize((256,256),),
    transforms.Normalize((0.5,),(0.5,))#just copied
])
root=r"C:\Users\saipr\OneDrive\Documents\PotatoPlants"
dataset=datasets.ImageFolder(root=root,transform=transform)#it will divide classes w.r.t to no.of folders inside it

# %%
train_size=int(0.9*(len(dataset)))
test_size=len(dataset)-train_size
train_data,test_data=random_split(dataset,[train_size,test_size])
train_loader=DataLoader(train_data,batch_size=64,shuffle=True)#train_data(that we are passing) should be tensor(my understanding)
test_loader=DataLoader(test_data,batch_size=64,shuffle=True)


# %%
class_names=dataset.classes#gives the  classes names
class_idx=dataset.class_to_idx

# %%
class CNN(nn.Module):
    def __init__(self,input_size,output_size):
        super().__init__()
        self.network=nn.Sequential(#intially I wrote self as nn
            nn.Conv2d(in_channels=3,out_channels=4,kernel_size=3,stride=1,padding=1),#these in channels are rbg colors of a colored photo
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=4,out_channels=8,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=8,out_channels=16,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Conv2d(in_channels=32,out_channels=32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2),
            nn.Flatten(),     
            nn.Linear(32*8*8,64),
            nn.ReLU(),
            nn.Linear(64,output_size)            
        )
    def forward(self,x):
        return self.network(x)

# %%
model=CNN(input_size=256*256,output_size=len(dataset.classes))
criterion=nn.CrossEntropyLoss()
optimiser=optim.Adam(model.parameters(),lr=0.001)

# %%
num_epochs=20
for epoch in range(num_epochs):
 for input,output in train_loader:
    pred_output=model(input)
    loss=criterion(pred_output,output)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
 print(f'Loss in Epoch [{epoch+1}/{num_epochs}]:{loss} ')
 if loss<0.01:
   break


# %%
def ClassificationReport(model,loader):
    y_test,y_pred=[],[]
    model.eval()
    with torch.no_grad():
        for input,output in loader:
            pred_output=model(input)
            y_test.extend(output.numpy())
            y_pred.extend(torch.argmax(pred_output,dim=1).numpy())
    model.train()
    print(classification_report(y_test,y_pred))
    
    for i in range(3):
        tc=sum(1 for y,x in zip(y_pred,y_test) if (y==i)&(x==i))
        ac=sum(1 for y in y_test if y==i)
        print(f'{list(class_idx.keys())[i]}: [{tc}/{ac}]')


# %%
print('classification report of training phase:')
ClassificationReport(model,train_loader)
print('Classification Report of testing phase:')
ClassificationReport(model,test_loader)

# %%
def accuracy(model,dataloader,classes):
    model.eval()
    with torch.no_grad():
        y_test,y_pred=[],[]
        for input,output in dataloader:
            pred_output=model(input)
            y_test.extend(output.numpy())
            y_pred.extend(torch.argmax(pred_output,dim=1).numpy())
        #classes=dataloader.classes
        cor,tot=[],[]
        for i in range(len(classes)):
            cor.append(sum(1 for j,k in zip(y_test,y_pred) if j==i and k==i))
            tot.append(sum (1 for j in y_test if j==i))
        print(f'accuracy report of {dataloader}:')
        for i,cl in enumerate(classes):
            print(f'{cl}:[{cor[i]}/{tot[i]}]')
        print(classification_report(y_test,y_pred))
    model.train()
print(accuracy(model,test_loader,class_names))
print(accuracy(model,train_loader,class_names))


# %%
pvp_root=r"C:\Users\saipr\Downloads\archive\PlantVillage\PlantVillage"
pvp_dataset=datasets.ImageFolder(root=pvp_root,transform=transform)
pvp_dataloader=DataLoader(pvp_dataset,batch_size=64,shuffle=True)
print(accuracy(model,pvp_dataloader,pvp_dataset.classes))



Loss in Epoch [1/20]:0.8213368058204651 
Loss in Epoch [2/20]:0.7611709237098694 
Loss in Epoch [3/20]:0.21643073856830597 
Loss in Epoch [4/20]:0.26407235860824585 
Loss in Epoch [5/20]:0.17381185293197632 
Loss in Epoch [6/20]:0.09788909554481506 
Loss in Epoch [7/20]:0.0157579705119133 
Loss in Epoch [8/20]:0.018425418063998222 
Loss in Epoch [9/20]:0.06150348484516144 
Loss in Epoch [10/20]:0.008474834263324738 
classification report of training phase:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       904
           1       0.97      0.98      0.98       893
           2       1.00      0.84      0.91       139

    accuracy                           0.98      1936
   macro avg       0.98      0.94      0.96      1936
weighted avg       0.98      0.98      0.98      1936

Potato___Early_blight: [897/904]
Potato___Late_blight: [878/893]
Potato___healthy: [117/139]
Classification Report of testing phase:
              precision  